In [1]:
import yaml
import os
import logging
import sys
import pandas as pd
import numpy as np
import pickle
import argparse

In [2]:
from modules.utils import load_cfg
from modules.data import PocketDataset, Dataloader
from modules.TrainIters import Pseq2SitesTrainIter
from modules.helpers import *

### 1. Load data

In [3]:
config = load_cfg("./configuration.yml")

2023-03-21 14:18:25,977 ERROR /data/BA_CM/tmp/Pseq2Sites/results/CV does not exist.
2023-03-21 14:18:25,978 ERROR /data/BA_CM/tmp/Pseq2Sites/results/prediction.tsv does not exist.


In [4]:
with open(config["paths"]["prot_feats"], "rb") as f:
    IDs, sequences, binding_sites, protein_feats = pickle.load(f)  

### 2. Make loader

In [5]:
dataset = PocketDataset(IDs, protein_feats, sequences)

In [6]:
loader = Dataloader(dataset, batch_size = config["train"]["batch_size"], shuffle = False)

### 3. Binding sites prediction

In [7]:
trainiter = Pseq2SitesTrainIter(config)

In [8]:
predicted_binding_sites = trainiter.run_test(loader, config["paths"]["model_path"])

/home/drugdesign/anaconda3/envs/cma/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


### 4. Get result

In [9]:
precision, recall, specificity, ACC, G_mean, F1_score, F2_score = get_results(binding_sites, convert_bs(predicted_binding_sites), sequences)

In [10]:
print(f"> COACH420 dataset results")
print(f"\tPrecision: {precision}, Recall: {recall}, Specficity: {specificity}, G-mean: {G_mean}, Accury: {ACC}, F1: {F1_score}, F2: {F2_score}")

> COACH420 dataset results
	Precision: 0.18, Recall: 0.95, Specficity: 0.76, G-mean: 0.85, Accury: 0.77, F1: 0.3, F2: 0.51
